<a href="https://colab.research.google.com/github/alisterpage/CHEM3580-Jupyter-Notebooks/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyscf geometric py3Dmol pythreejs -q 2>&1 >/dev/null
import pyscf
from pyscf import gto, scf, tools, lo, lib, dft
#from pyscf.geomopt.berny_solver import optimize
from pyscf.geomopt.geometric_solver import optimize
from geometric.molecule import Molecule
import py3Dmol
from google.colab import output
output.enable_custom_widget_manager()
import numpy as np
from __future__ import print_function
import sys

In [ ]:
import ipywidgets as widgets

dihedral_textbox = widgets.FloatText(
    value=0.0,
    description='Dihedral Angle (degrees):',
    disabled=False
)

display(dihedral_textbox)

# Get the value entered by the user in the widget and store it as a float variable called dihedral


In [ ]:
#define molecule
initial_dihedral = dihedral_textbox.value
mol = gto.Mole()
#mol.atom = '''
#    H
#    H 1 1.8
#    H 2 1.8 1 60.0
#    C 1 1.1 2 35.2 3 {}
#'''.format(initial_dihedral)


mol.atom = '''
C
C  1  1.5288
C  2  1.5369  1  a3
C  3  1.5288  2  a4  1  d4
H  2  r5  1  a5  3  d5
H  2  r6  1  a6  3  d6
H  3  r7  2  a7  1  d7
H  3  r8  2  a8  1  d8
H  4  r9  3  a9  2  d9
H  4  r10  3  a10  2  d10
H  4  r11  3  a11  2  d11
H  1  r12  2  a12  3  d12
H  1  r13  2  a13  3  d13
H  1  r14  2  a14  3  d14
Variables:
r2= 
r3= 
a3= 113.35
r4= 1.5288
a4= 113.35
d4=  64.43
r5= 1.1116
a5= 107.90
d5= 240.40
r6= 1.1117
a6= 109.51
d6= 122.95
r7= 1.1117
a7= 109.78
d7= 301.63
r8= 1.1116
a8= 108.04
d8= 183.95
r9= 1.1089
a9= 111.26
d9= 296.51
r10= 1.1104
a10= 109.33
d10= 177.48
r11= 1.1107
a11= 110.27
d11=  58.12
r12= 1.1107
a12= 110.27
d12=  58.12
r13= 1.1089
a13= 111.26
d13= 296.50
r14= 1.1104
a14= 109.33
d14= 177.47
'''

#define model chemistry & calculation 
mol.basis = '6-31g(d)'
mol.spin=1;
mol.build();
mf = dft.ROKS(mol)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Optimize the geometry
text = "$freeze\ndihedral 1 2 3 4\n"
with open("constraints.txt", "w") as f:
    f.write(text)
params = {"constraints": "constraints.txt",}
mol_eq = optimize(mf, **params)

#print(mol_eq.atom_coords())

# Get the optimized energy
mf = dft.ROKS(mol_eq)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Create a temporary XYZ file with the optimized coordinates
symbols = [atom[0] for atom in mol_eq.atom];
coords = mol_eq.atom_coords();
with open("temp.xyz", "w") as f:
    f.write("%d\n" % len(symbols))
    f.write("Generated by PySCF and Geometric\n")
    for i in range(len(symbols)):
        f.write("%s %f %f %f\n" % (symbols[i], coords[i,0], coords[i,1], coords[i,2]))

# Convert the PySCF object to a `Molecule` object using the XYZ file and store dihedral angle.
mol_geom = Molecule("temp.xyz");
dihedral=Molecule.measure_dihedrals(mol_geom,0,1,2,3);